# El Yapımı Standartlaştırıcı

🧑🏻‍🏫 Bu challenge'da *kendi* StandardScaler'ımızı oluşturacağız. Neden mi diye merak ediyorsunuz? Sorduğunuz iyi oldu!

🎯 Bu egzersizin hedefleri:
- `durumsuz transformatörler` ile `durumsal transformatörler` arasındaki farkı anlamak
- `FeatureUnion` ile çalışmak

## (1) 📚 Durumsuz Transformatör vs. Durumsal Transformatör

🔢 Aşağıdaki eğitim seti ve test setini göz önünde bulunduralım...

In [1]:
import numpy as np
import pandas as pd

X_train = pd.DataFrame({
    'A': {0: 1, 1: 2, 2: 3},
    'B': {0: 4, 1: 5, 2: 6},
    'C': {0: 7, 1: 8, 2: 9}
})

print("This is the training dataset:")
display(X_train)

X_test = pd.DataFrame({
    'A': {0: 1, 1: 2, 2: 3},
    'B': {0: 2, 1: 3, 2: 4},
    'C': {0: 3, 1: 4, 2: 10}
})

print("This is the test dataset:")
display(X_test)

This is the training dataset:


,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


This is the test dataset:


,A,B,C
0,1,2,3
1,2,3,4
2,3,4,10


🛠 ...ve aşağıdaki union'u:
- özellikleri ölçeklendirir
- diğer (ölçeklenmemiş) özelliklerin ortalamasını alan yeni bir özellik oluşturur

In [2]:
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import make_pipeline, make_union

standard_scaler = StandardScaler()
feature_averager = FunctionTransformer(lambda df: pd.DataFrame(1/3 * (df["A"] + df["B"] + df["C"])))

pipeline = make_union(standard_scaler, feature_averager)
pipeline

FeatureUnion(transformer_list=[('standardscaler', StandardScaler()),
                               ('functiontransformer',
                                FunctionTransformer(func=<function <lambda> at 0x71ac4babde40>))])

Şimdi:
- pipeline'ı eğitim setine uydur
- hem eğitim setini hem de test setini dönüştür

In [3]:
pipeline.fit(X_train)

FeatureUnion(transformer_list=[('standardscaler', StandardScaler()),
                               ('functiontransformer',
                                FunctionTransformer(func=<function <lambda> at 0x71ac4babde40>))])

In [4]:
X_train_transformed = pd.DataFrame(pipeline.transform(X_train))
X_train_transformed

,0,1,2,3
0,-1.224745,-1.224745,-1.224745,4.0
1,0.000000,0.000000,0.000000,5.0
2,1.224745,1.224745,1.224745,6.0


In [5]:
X_test_transformed = pd.DataFrame(pipeline.transform(X_test))
X_test_transformed

,0,1,2,3
0,-1.224745,-3.674235,-6.123724,2.000000
1,0.000000,-2.449490,-4.898979,3.000000
2,1.224745,-1.224745,2.449490,5.666667


👨🏻‍🏫 `StandardScaler` ve `FunctionTransformer`'ın temelde nasıl farklı olduğuna dikkat edin ❗️

Pipeline'ı uydurduğumuzda ve eğitim seti ile test setini dönüştürmek için kullandığımızda:

* **`FunctionTransformer (feature_averager)`**:
    * *.fit()* sırasında hiçbir şey "öğrenmedi"
    * sadece **durumsuz bir dönüşüm** gerçekleştirdi: $ \large (X_1, X_2, X_3) \rightarrow \frac{(X_1 + X_2 + X_3)}{3}$


* **`StandardScaler`**:
    * *.fit()* sırasında $\mu_{\color{blue}{train}}$ ve $\sigma
   _{\color{blue}{train}}$ "öğrendi"
    * hem eğitim setinde hem de test setinde öğrendiği bu değerleri kullanarak **durumsal bir dönüşüm** gerçekleştirdi:
        * $ \large X_{\color{blue}{train-scaled}} =  \frac{X_{\color{blue}{train}} -\mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}$
        * $ \large X_{\color{red}{test-scaled}} =  \frac{X_{\color{red}{test}} -\mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}$

## (2) 💻 Kendi durumsal transformatörünüzü oluşturun

🤔 Kendi **durumsal özel transformatörümüzü** kodlamak istersek ne olur?

💪 Kendi sınıfımızı kodlayabiliriz!

### (2.1) 💻 Özel Standartlaştırıcı

❓ **Sorular: Kendi sınıfınızı kodlamak** ❓

1. Kendi `CustomStandardizer` sınıfınızı kodlayın
    * Scikit Learn'deki `StandardScaler` ile tam olarak aynı şekilde davranmalı, bu şu anlama gelir:
        * $\mu_{\color{blue}{train}}$ ve $\sigma
   _{\color{blue}{train}}$ hesaplayan ("öğrenen") bir `.fit()` metodu
        * ve bir `.transform()` metodu.


2. Bunu `X_train` üzerinde uydur

3. Hem `X_train` hem de `X_test`'i dönüştür

4. Doğru yaptığınızdan emin olmak için `CustomStandardizer`ınızı Scikit Learn'deki `StandardScaler` ile karşılaştır!

In [6]:
#########################################
# 1 - Code the CustomStandardizer Class #
#########################################

# TransformerMixin inheritance is used to create fit_transform() method from fit() and transform()
from sklearn.base import TransformerMixin, BaseEstimator

# $DELETE_BEGIN
# Bonus: allows us to raise a NotFittedError when one calls the transform method before fitting the instance
from sklearn.exceptions import NotFittedError
# $DELETE_END

class CustomStandardizer(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        '''
        Stores what needs to be stored as instance attributes.
        ReturnS "self" to allow chaining fit and transform.
        '''
        # $CHALLENGIFY_BEGIN
        self.means = X.mean()
        self.stds = X.std(ddof=0)

        # Return self to allow chaining & fit_transform
        return self
        # $CHALLENGIFY_END

    def transform(self, X, y=None):
        # $CHALLENGIFY_BEGIN
        if not (hasattr(self, "means") and hasattr(self, "stds")):
            raise NotFittedError("This CustomStandardScaler instance is not fitted yet. Call 'fit' with the appropriate arguments before using this estimator.")

        # Standardization
        standardized_features = (X - self.means) / self.stds

        return standardized_features
        # $CHALLENGIFY_END

    # $DELETE_BEGIN
    def inverse_transform(self, X, y=None):
        if not (hasattr(self, "means") and hasattr(self, "stds")):
            raise NotFittedError("This CustomStandardScaler instance is not fitted yet. Call 'fit' with the appropriate arguments before using this estimator.")

        return X * self.stds + self.means
    # $DELETE_END

In [7]:
#########################################
# 2 - Fit the CustomStandardizer Class  #
#########################################

custom_standardizer = CustomStandardizer()
custom_standardizer.fit(X_train)

CustomStandardizer()

In [8]:
#########################################
# 3 - Transform                         #
#########################################

# $CHALLENGIFY_BEGIN

X_train_transformed = custom_standardizer.transform(X_train)
X_test_transformed = custom_standardizer.transform(X_test)

print("X_train_transformed:")
display(X_train_transformed)

print("X_test_transformed:")
X_test_transformed

# $CHALLENGIFY_END

X_train_transformed:


,A,B,C
0,-1.224745,-1.224745,-1.224745
1,0.000000,0.000000,0.000000
2,1.224745,1.224745,1.224745


X_test_transformed:


,A,B,C
0,-1.224745,-3.674235,-6.123724
1,0.000000,-2.449490,-4.898979
2,1.224745,-1.224745,2.449490


🧪 **Kodunuzu test edin**

In [9]:
from nbresult import ChallengeResult

tmp = CustomStandardizer()
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult(
    'standardizer',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/wit/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/wit/code/S17D1-S-Data-hand-made-standardizer/tests
plugins: typeguard-4.4.2, anyio-4.8.0
collecting ... collected 1 item

test_standardizer.py::TestStandardizer::test_solution PASSED             [100%]

============================== 1 passed in 0.34s ===============================


💯 You can commit your code:

git add tests/standardizer.pickle

git commit -m 'Completed standardizer step'

git push origin master



<details>
<summary>💡 <i>İpuçları</i> (🧪 eğer yukarıdaki testler sadece küçük bir farkla başarısız oluyorsa) </summary>

* `np.std()` ve `pd.std` metodları arasında küçük bir fark olduğuna dikkat edin!
    
* Bu [Stack Overflow gönderisi](https://stackoverflow.com/questions/44220290/sklearn-standardscaler-result-different-to-manual-result) yardımcı olabilir 😉
      
</details>

<details>
<summary>💡 <i>Çözüm</i></summary>

`pd.std` metodunu kullanırsanız, scikit-learn'ün StandardScaler davranışını taklit etmek için `ddof=0` parametresini de kullanmanız gerekir.      
</details>

### (2.2) 💻 Ters Dönüşüm

❓ **Soru (Ters Dönüşüm)** ❓

Scikit Learn'deki _StandardScaler_'ın ölçeklenmemiş veri setine geri dönmenize yardımcı olan bir [`.inverse_transform()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler.inverse_transform) metodu vardır.

1. `CustomStandardizer` sınıfınıza geri dönün ve kendi `.inverse_transform()` metodunuzu uygulayın.

2. Bunu ölçeklendirilmiş eğitim setinizde ve ölçeklendirilmiş test setinizde deneyin.

In [10]:
# YOUR CODE HERE

In [11]:
# Try your inverse transform and run the test down below to make sure you coded it correctly

# $DELETE_BEGIN
custom_scaler = CustomStandardizer().fit(X_train)

X_train_transformed = custom_scaler.transform(X_train)
X_train_inverse_transformed = custom_scaler.inverse_transform(X_train_transformed)

X_test_transformed = custom_scaler.transform(X_test)
X_test_inverse_transformed = custom_scaler.inverse_transform(X_test_transformed)

print("X_train_transformed:")
display(X_train_transformed)

print("X_train_inverse_transformed (back to the original):")
display(X_train_inverse_transformed)

print("X_test_transformed:")
display(X_test_transformed)

print("X_test_inverse_transformed (back to the original):")
display(X_test_inverse_transformed)

# $DELETE_END

X_train_transformed:


,A,B,C
0,-1.224745,-1.224745,-1.224745
1,0.000000,0.000000,0.000000
2,1.224745,1.224745,1.224745


X_train_inverse_transformed (back to the original):


,A,B,C
0,1.0,4.0,7.0
1,2.0,5.0,8.0
2,3.0,6.0,9.0


X_test_transformed:


,A,B,C
0,-1.224745,-3.674235,-6.123724
1,0.000000,-2.449490,-4.898979
2,1.224745,-1.224745,2.449490


X_test_inverse_transformed (back to the original):


,A,B,C
0,1.0,2.0,3.0
1,2.0,3.0,4.0
2,3.0,4.0,10.0


🧪 **Kodunuzu test edin**

In [12]:
assert np.allclose(X_train_inverse_transformed, X_train)
assert np.allclose(X_test_inverse_transformed, X_test)

### (2.3) 💻 Tam özel pipeline!

💪 Scikit-Learn'ün `StandardScaler`'ını kopyalamayı başardık.

🌶 Biraz baharatlayalım!

❓ **Soru: önceki `CustomStandardizer` özel transformatörünü bir küçültme faktörü ile geliştirin** ❓


`CustomStandardizer(shrink_factor = 1)` sınıfı, $\sigma_{\color{blue}{train}}$'e orantılı daha güçlü bir ölçekleme gerçekleştirmek için ek bir argüman almalı 👇:
- $ \large X_{\color{blue}{train-scaled}} =  (\frac{X_{\color{blue}{train}} -\mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}) \times \frac{1}{shrink factor}$
- $ \large X_{\color{red}{test-scaled}} =  (\frac{X_{\color{red}{test}} -\mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}) \times \frac{1}{shrink factor}$


In [13]:
###################################
# Custom Standardizer             #
###################################

# $DELETE_BEGIN
# Bonus: allows us to raise a NotFittedError when one calls the transform method before fitting the instance
from sklearn.exceptions import NotFittedError
# $DELETE_END

class CustomStandardizer(TransformerMixin, BaseEstimator):

    def __init__(self, shrink_factor = 1):
        # $CHALLENGIFY_BEGIN
        self.shrink_factor = shrink_factor
        # $CHALLENGIFY_END

    def fit(self, X, y=None):
        '''
        Stores what needs to be stored as instance attributes.
        Returns "self" to allow chaining fit and transform.
        '''
        # $CHALLENGIFY_BEGIN
        self.means = X.mean()
        self.stds = X.std(ddof=0)

        # Return self to allow chaining & fit_transform
        return self
        # $CHALLENGIFY_END

    def transform(self, X, y=None):
        # $CHALLENGIFY_BEGIN
        if not (hasattr(self, "means") and hasattr(self, "stds")):
            raise NotFittedError("This CustomStandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.")

        # Standardization
        standardized_features = (X - self.means) / self.stds / self.shrink_factor

        return standardized_features
        # $CHALLENGIFY_END

    def inverse_transform(self, X, y=None):
        # $CHALLENGIFY_BEGIN
        if not (hasattr(self, "means") and hasattr(self, "stds")):
            raise NotFittedError("This CustomStandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.")

        return X * self.shrink_factor * self.stds + self.means
        # $CHALLENGIFY_END

🧪 **Yeni `CustomStandardizer` özel transformatörünüzü test edin** (`shrink_factor = 2`) `X_train` üzerinde uydurarak ve hem `X_train` hem de `X_test`'i dönüştürerek ve dönüştürülen DataFrame'leri `X_train_transformed` ve `X_test_transformed`'a kaydedin

In [14]:
custom_scaler = CustomStandardizer(shrink_factor=2).fit(X_train)

print("X_train_transformed and shrunk by 1/2 = 50%")
X_train_transformed = custom_scaler.transform(X_train)
display(X_train_transformed)

print("X_test_transformed and shrunk by 1/2 = 50%")
X_test_transformed = custom_scaler.transform(X_test)
display(X_test_transformed)


X_train_transformed and shrunk by 1/2 = 50%


,A,B,C
0,-0.612372,-0.612372,-0.612372
1,0.000000,0.000000,0.000000
2,0.612372,0.612372,0.612372


X_test_transformed and shrunk by 1/2 = 50%


,A,B,C
0,-0.612372,-1.837117,-3.061862
1,0.000000,-1.224745,-2.449490
2,0.612372,-0.612372,1.224745


🧪 **Kodunuzu test edin**

In [15]:
from nbresult import ChallengeResult

tmp = CustomStandardizer(shrink_factor=2).fit(X_train)
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult(
    'new_standardizer',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/wit/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/wit/code/S17D1-S-Data-hand-made-standardizer/tests
plugins: typeguard-4.4.2, anyio-4.8.0
collecting ... collected 1 item

test_new_standardizer.py::TestNewStandardizer::test_solution PASSED      [100%]

============================== 1 passed in 0.38s ===============================


💯 You can commit your code:

git add tests/new_standardizer.pickle

git commit -m 'Completed new_standardizer step'

git push origin master



In [16]:
# Run the following cells to ensure you got the right transformations
# truth_train = np.array([
#     [-0.612372, -0.612372, -0.612372],
#     [0.000000, 0.000000, 0.000000],
#     [0.612372, 0.612372, 0.612372]
# ])
# truth_test = np.array([
#     [-0.612372, -1.837117, -3.061862],
#     [ 0.        , -1.224745, -2.449490],
#     [ 0.612372, -0.612372,  1.224745]])

In [17]:
# # Asserts
# np.allclose(X_train_transformed, truth_train)

In [18]:
# # Assert - Test
# np.allclose(X_test_transformed, truth_test)

❓ **Soru: önceki `FeatureAverager` özel transformatörünü "ayarlayın"** ❓

Bu değiştirilmiş `FeatureAverager()` sınıfı:
- hâlâ üç farklı özelliğin ortalamasını hesaplar...
- ...ve sonra sonucu her satır için maksimum değere böler

_Not: Bu işlemi yorumlamaya çalışmayın. Gerçek bir anlamı yok, sadece özel sınıf kodlama becerilerimizi geliştirmek istiyoruz._

$$(X_1, X_2, X_3) \rightarrow \frac{mean(X_1 + X_2 + X_3)}{max(X_1, X_2, X_3)}$$

_Not_: Burada durumsal mı yoksa durumsuz bir dönüşümle mi uğraştığımızı dikkatli düşünün (yani dönüştürmeden önce tam eğitim verimiz hakkında bilgi öğrenmemiz gerekiyor mu - *durumsal* StandardScaler'ımızda yaptığımız gibi - yoksa yukarıdaki *durumsuz* `feature_averager`'ımız gibi mi çalışıyor?). İlki ise, `.fit()` için bir şeyler yazmamız gerekir - değilse yapmamız gerekmez!


In [19]:
###################################
# Feature Averager                #
###################################

class FeatureAverager(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        '''
        If needed, this method will store information instance attributes.
        Returns "self".
        '''
        # $CHALLENGIFY_BEGIN
        # Since our feature averager is a stateless transformation,
        # there is nothing to fit!
        return self
        # $CHALLENGIFY_END

    def transform(self, X, y=None):
        # $CHALLENGIFY_BEGIN
        features_mean = X.mean(axis="columns")
        max_factor = X.max(axis = "columns")
        feature_averager = (features_mean) / max_factor

        return pd.DataFrame(feature_averager)
        # $CHALLENGIFY_END

🧪 **`FeatureAverager` özel transformatörünüzü test edin** `X_train` üzerinde uydurarak ve hem `X_train` hem de `X_test`'i dönüştürerek

In [20]:
# $CHALLENGIFY_BEGIN
custom_feature_averager = FeatureAverager().fit(X_train)

X_train_transformed = custom_feature_averager.transform(X_train)
X_test_transformed = custom_feature_averager.transform(X_test)

print("X_train averaged divided by the maximum value on each row:")
display(X_train_transformed)

print("X_test averaged divided by the maximum value on each row:")
display(X_test_transformed)
# $CHALLENGIFY_END

X_train averaged divided by the maximum value on each row:


,0
0,0.571429
1,0.625000
2,0.666667


X_test averaged divided by the maximum value on each row:


,0
0,0.666667
1,0.750000
2,0.566667


In [21]:
from nbresult import ChallengeResult

tmp = FeatureAverager()
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult(
    'feature_averager',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/wit/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/wit/code/S17D1-S-Data-hand-made-standardizer/tests
plugins: typeguard-4.4.2, anyio-4.8.0
collecting ... collected 1 item

test_feature_averager.py::TestFeatureAverager::test_solution PASSED      [100%]

============================== 1 passed in 0.49s ===============================


💯 You can commit your code:

git add tests/feature_averager.pickle

git commit -m 'Completed feature_averager step'

git push origin master



❓ **Soru (Feature Union)** ❓

1. Hem `CustomStandardizer` hem de `FeatureAverager` kullanarak bir `FeatureUnion` pipeline'ı oluşturun ve bunu `pipeline` adlı bir değişkende saklayın.

2. Pipeline'ı `X_train`'e uydur ve hem `X_train` hem de `X_test`'i dönüştür (`shrink_factor = 3`)

3. Bu challenge'ın son testini geçtiğinizden emin olun.

In [22]:
#####################
# 1 - Feature Union #
#####################

# $CHALLENGIFY_BEGIN
custom_standardizer = CustomStandardizer(shrink_factor=3)
custom_feature_averager = FeatureAverager()

pipeline = make_union(custom_standardizer, custom_feature_averager)
pipeline
# $CHALLENGIFY_END

FeatureUnion(transformer_list=[('customstandardizer',
                                CustomStandardizer(shrink_factor=3)),
                               ('featureaverager', FeatureAverager())])

In [23]:
#########################
# 2 - Fit and Transform #
#########################

# $CHALLENGIFY_BEGIN
pipeline.fit(X_train)

X_train_transformed = pd.DataFrame(pipeline.transform(X_train))
X_test_transformed = pd.DataFrame(pipeline.transform(X_test))

print("X_train_transformed with the custom pipeline:")
display(X_train_transformed)

print("X_test_transformed with the custom pipeline:")
display(X_test_transformed)
# $CHALLENGIFY_END

X_train_transformed with the custom pipeline:


,0,1,2,3
0,-0.408248,-0.408248,-0.408248,0.571429
1,0.000000,0.000000,0.000000,0.625000
2,0.408248,0.408248,0.408248,0.666667


X_test_transformed with the custom pipeline:


,0,1,2,3
0,-0.408248,-1.224745,-2.041241,0.666667
1,0.000000,-0.816497,-1.632993,0.750000
2,0.408248,-0.408248,0.816497,0.566667


In [24]:
from nbresult import ChallengeResult

tmp = pipeline
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))



result = ChallengeResult(
    'feature_union_custom_transformers',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/wit/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/wit/code/S17D1-S-Data-hand-made-standardizer/tests
plugins: typeguard-4.4.2, anyio-4.8.0
collecting ... collected 1 item

test_feature_union_custom_transformers.py::TestFeatureUnionCustomTransformers::test_solution PASSED [100%]

============================== 1 passed in 0.73s ===============================


💯 You can commit your code:

git add tests/feature_union_custom_transformers.pickle

git commit -m 'Completed feature_union_custom_transformers step'

git push origin master



🏁 Tebrikler! Kendi Transformatörünüzü nasıl oluşturacağınızı keşfettiniz!

💾 Notebook'unuzu `git add/commit/push` etmeyi unutmayın...

🚀 ... ve bir sonraki challenge'a geçin!